# Skip-Gram+层次Softmax数据处理模块

In [1]:
import numpy as np
from collections import deque
from huffman_tree import HuffmanTree

In [2]:
input_file = open("./data/text8.txt","r",encoding="utf-8")

In [3]:
def _init_dict(min_count):
    word_count_sum = 0
    sentence_count = 0
    word2id_dict = dict()
    id2word_dict = dict()
    wordid_frequency_dict = dict()
    word_freq = dict()
    for line in input_file:
        line = line.strip().split()
        word_count_sum +=len(line)
        sentence_count +=1
        for i,word in enumerate(line):
            if i%1000000==0:
                print (i,len(line))
            if word_freq.get(word)==None:
                word_freq[word] = 1
            else:
                word_freq[word] += 1
    for i,word in enumerate(word_freq):
        if i % 100000 == 0:
            print(i, len(word_freq))
        if word_freq[word]<min_count:
            word_count_sum -= word_freq[word]
            continue
        word2id_dict[word] = len(word2id_dict)
        id2word_dict[len(id2word_dict)] = word
        wordid_frequency_dict[len(word2id_dict)-1] = word_freq[word]
    word_count =len(word2id_dict)
    return word2id_dict,id2word_dict,wordid_frequency_dict

In [4]:
word2id_dict,id2word_dict,wordid_frequency_dict = _init_dict(20)

0 17005207
1000000 17005207
2000000 17005207
3000000 17005207
4000000 17005207
5000000 17005207
6000000 17005207
7000000 17005207
8000000 17005207
9000000 17005207
10000000 17005207
11000000 17005207
12000000 17005207
13000000 17005207
14000000 17005207
15000000 17005207
16000000 17005207
17000000 17005207
0 253854
100000 253854
200000 253854


In [7]:
len(word2id_dict)

31892

In [5]:
def get_wordId_list():
    input_file = open("./data/text8.txt", encoding="utf-8")
    sentence = input_file.readline()
    wordId_list = []  # 一句中的所有word 对应的 id
    sentence = sentence.strip().split(' ')
    for i,word in enumerate(sentence):
        if i%1000000==0:
            print (i,len(sentence))
        try:
            word_id = word2id_dict[word]
            wordId_list.append(word_id)
        except:
            continue
    return wordId_list

In [6]:
wordId_list = get_wordId_list()

0 17005207
1000000 17005207
2000000 17005207
3000000 17005207
4000000 17005207
5000000 17005207
6000000 17005207
7000000 17005207
8000000 17005207
9000000 17005207
10000000 17005207
11000000 17005207
12000000 17005207
13000000 17005207
14000000 17005207
15000000 17005207
16000000 17005207
17000000 17005207


In [8]:
wordId_list

[20592,
 4594,
 10730,
 15282,
 1861,
 19056,
 1236,
 28503,
 27257,
 5526,
 29373,
 22109,
 214,
 2299,
 25280,
 11012,
 1338,
 19056,
 11012,
 8719,
 22890,
 22969,
 11012,
 25968,
 19056,
 11012,
 15372,
 22890,
 5149,
 11012,
 1861,
 19872,
 25723,
 27257,
 8992,
 15282,
 15311,
 30427,
 26154,
 4718,
 26460,
 9725,
 1141,
 27257,
 28113,
 7565,
 26154,
 6226,
 11012,
 269,
 19056,
 13146,
 8320,
 18710,
 4915,
 18433,
 21746,
 15937,
 10730,
 15282,
 12446,
 12164,
 14346,
 6293,
 31556,
 21555,
 11012,
 24955,
 20592,
 19872,
 22737,
 31479,
 11012,
 9415,
 22460,
 9480,
 16171,
 3724,
 20592,
 10730,
 15282,
 25603,
 21513,
 19872,
 11012,
 3373,
 1141,
 30549,
 23185,
 11614,
 22969,
 12370,
 15500,
 2359,
 16413,
 17230,
 23185,
 1186,
 9385,
 19056,
 19277,
 3904,
 7565,
 20592,
 4915,
 15023,
 26154,
 10178,
 25756,
 23984,
 1141,
 20875,
 11012,
 27486,
 19056,
 5193,
 9836,
 30322,
 11012,
 102,
 11012,
 24955,
 26706,
 10730,
 16526,
 21555,
 5685,
 8320,
 14254,
 6941,
 

In [9]:
def get_batch_pairs(batch_size,window_size,index,word_pairs_queue):
    while len(word_pairs_queue) < batch_size:
        for _ in range(1000):
            if index == len(wordId_list):
                index = 0
            for i in range(max(index - window_size, 0), min(index + window_size + 1,len(wordId_list))):
                wordId_w = wordId_list[index]
                wordId_v = wordId_list[i]
                if index == i:  # 上下文=中心词 跳过
                    continue
                word_pairs_queue.append((wordId_w, wordId_v))
            index+=1
    result_pairs = []  # 返回mini-batch大小的正采样对
    for _ in range(batch_size):
        result_pairs.append(word_pairs_queue.popleft())
    return result_pairs

In [10]:
index = 0
word_pairs_queue = deque()
result_pairs = get_batch_pairs(32,3,index,word_pairs_queue)

In [11]:
result_pairs

[(20592, 4594),
 (20592, 10730),
 (20592, 15282),
 (4594, 20592),
 (4594, 10730),
 (4594, 15282),
 (4594, 1861),
 (10730, 20592),
 (10730, 4594),
 (10730, 15282),
 (10730, 1861),
 (10730, 19056),
 (15282, 20592),
 (15282, 4594),
 (15282, 10730),
 (15282, 1861),
 (15282, 19056),
 (15282, 1236),
 (1861, 4594),
 (1861, 10730),
 (1861, 15282),
 (1861, 19056),
 (1861, 1236),
 (1861, 28503),
 (19056, 10730),
 (19056, 15282),
 (19056, 1861),
 (19056, 1236),
 (19056, 28503),
 (19056, 27257),
 (1236, 15282),
 (1236, 1861)]

In [12]:
huffman_tree = HuffmanTree(wordid_frequency_dict)  # 霍夫曼树
huffman_pos_path, huffman_neg_path = huffman_tree.get_all_pos_and_neg_path()
def get_pairs(pos_pairs):
    neg_word_pair = []
    pos_word_pair = []
    for pair in pos_pairs:
        pos_word_pair += zip([pair[0]] * len(huffman_pos_path[pair[1]]), huffman_pos_path[pair[1]])
        neg_word_pair += zip([pair[0]] * len(huffman_neg_path[pair[1]]), huffman_neg_path[pair[1]])
    return pos_word_pair, neg_word_pair

In [14]:
pos_word_pair, neg_word_pair = get_pairs(result_pairs)

In [15]:
pos_word_pair

[(20592, 63782),
 (20592, 63778),
 (20592, 63773),
 (20592, 63763),
 (20592, 63745),
 (20592, 63714),
 (20592, 63656),
 (20592, 63024),
 (20592, 62408),
 (20592, 61356),
 (20592, 59660),
 (20592, 57116),
 (20592, 63782),
 (20592, 63778),
 (20592, 63742),
 (20592, 63782),
 (20592, 63781),
 (20592, 63779),
 (4594, 63782),
 (4594, 63781),
 (4594, 63774),
 (4594, 63717),
 (4594, 63662),
 (4594, 63388),
 (4594, 63060),
 (4594, 61458),
 (4594, 59827),
 (4594, 57346),
 (4594, 63782),
 (4594, 63778),
 (4594, 63742),
 (4594, 63782),
 (4594, 63781),
 (4594, 63779),
 (4594, 63780),
 (4594, 63777),
 (4594, 63736),
 (4594, 63271),
 (10730, 63782),
 (10730, 63781),
 (10730, 63774),
 (10730, 63717),
 (10730, 63662),
 (10730, 63388),
 (10730, 63060),
 (10730, 61458),
 (10730, 59827),
 (10730, 57346),
 (10730, 63782),
 (10730, 63778),
 (10730, 63773),
 (10730, 63763),
 (10730, 63745),
 (10730, 63714),
 (10730, 63656),
 (10730, 63024),
 (10730, 62408),
 (10730, 61356),
 (10730, 59660),
 (10730, 57116),


In [16]:
neg_word_pair

[(20592, 63781),
 (20592, 63556),
 (20592, 63369),
 (20592, 63781),
 (20592, 63773),
 (20592, 63762),
 (20592, 63709),
 (20592, 63775),
 (20592, 63766),
 (20592, 63749),
 (4594, 63779),
 (4594, 63765),
 (4594, 63747),
 (4594, 63566),
 (4594, 62470),
 (4594, 53850),
 (4594, 63781),
 (4594, 63773),
 (4594, 63762),
 (4594, 63709),
 (4594, 63775),
 (4594, 63766),
 (4594, 63749),
 (4594, 63782),
 (4594, 63771),
 (4594, 63759),
 (4594, 63698),
 (4594, 63628),
 (4594, 63505),
 (4594, 62845),
 (10730, 63779),
 (10730, 63765),
 (10730, 63747),
 (10730, 63566),
 (10730, 62470),
 (10730, 53850),
 (10730, 63781),
 (10730, 63556),
 (10730, 63369),
 (10730, 63775),
 (10730, 63766),
 (10730, 63749),
 (10730, 63782),
 (10730, 63771),
 (10730, 63759),
 (10730, 63698),
 (10730, 63628),
 (10730, 63505),
 (10730, 62845),
 (10730, 63779),
 (10730, 63774),
 (10730, 63764),
 (15282, 63779),
 (15282, 63765),
 (15282, 63747),
 (15282, 63566),
 (15282, 62470),
 (15282, 53850),
 (15282, 63781),
 (15282, 63556),
